In [2]:
# importing libraries
import numpy as np
import pandas as pd
from PIL import Image
import torch
import tensorflow as tf
import torchvision
from torchvision import transforms
from torchvision import datasets, transforms, models
from torch.utils.data import random_split
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import pickle as pkl
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import sklearn
import seaborn as sns
import cv2
import skimage.exposure
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
import PIL

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
RezNetTransform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, transform = RezNetTransform):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        filename = str(self.df.id[index])+'.jpg'
        label = self.df.stable[index]
        image = PIL.Image.open(os.path.join(self.images_folder, filename))
        sample = {'image': image, 'stable': label}
        if self.transform:
            sample['image'] = self.transform(sample['image'])
        return sample

In [6]:
dataset = CustomDataset("blocks-labels.csv", './data')

In [7]:
train_size = int(0.8 * len(dataset))
test_size = 51276 - train_size
train, test = torch.utils.data.random_split(dataset, [train_size, test_size])

In [8]:
trainloader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test, batch_size=4, shuffle=True, num_workers=2)

In [5]:
resnet = models.resnet18(pretrained=True)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 2)
resnet = resnet.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alex_net.parameters(), lr=0.01, momentum=0.9)
print(criterion, optimizer)

losses = []
for epoch in range(1):  # loop over the dataset
    running_loss = 0.0
    for i, data in enumerate(train_loader_1, 0):
        # get the inputs; data is a list of [inputs, labels]
        # zero the parameter gradients
        inputs, labels = data['image'], data['stable']
        optimizer.zero_grad()
        # forward + backward + optimize
        output = alex_net(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:   
            print(loss.item())
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
        epoch_loss = running_loss / len(train_loader_1)
        losses.append(epoch_loss)
print('Finished Training of AlexNet')